# 2 Derivatives and Gradients

## 2.1 Derivatives

In [1]:
using SymEngine
@vars x; # define x as a symbolic variable
f = x^2 + x/2 - sin(x)/x;
diff(f, x)

1/2 + 2*x + sin(x)/x^2 - cos(x)/x

## 2.3 Numerical Differentiation

### 2.3.1 Finite Difference Methods

In [2]:
diff_forward(f, x; h=sqrt(eps(Float64))) = (f(x+h) - f(x))/h
diff_central(f, x; h=cbrt(eps(Float64))) = (f(x+h/2) - f(x-h/2))/h
diff_backward(f, x; h=sqrt(eps(Float64))) = (f(x) - f(x-h))/h

diff_backward (generic function with 1 method)

### 2.3.2 Complex Step Method

In [3]:
diff_complex(f, x; h=1e-20) = imag(f(x + h*im)) / h

diff_complex (generic function with 1 method)

In [4]:
f = x -> sin(x^2);
v = f(π/2 + 0.001im);
f(π/2)

0.6242659526396992

In [5]:
real(v) # f(x)

0.6242698144866649

In [6]:
imag(v)/0.001 # f'(x)

-2.4542516170381785

## 2.4 Automatic Differentiation

### 2.4.1 Forward Accumulation

In [1]:
struct Dual
    v
    ∂
end

In [2]:
Base.:+(a::Dual, b::Dual) = Dual(a.v + b.v, a.∂ + b.∂)
Base.:*(a::Dual, b::Dual) = Dual(a.v * b.v, a.v*b.∂ + b.v*a.∂)
Base.log(a::Dual) = Dual(log(a.v), a.∂/a.v)
function Base.max(a::Dual, b::Dual)
    v = max(a.v, b.v)
    ∂ = a.v > b.v ? a.∂ : a.v < b.v ? b.∂ : NaN
    return Dual(v, ∂)
end
function Base.max(a::Dual, b::Int)
    v = max(a.v, b)
    ∂ = a.v > b ? a.∂ : a.v < b ? 0 : NaN
    return Dual(v, ∂)
end

In [4]:
using ForwardDiff
a = ForwardDiff.Dual(3,1);
b = ForwardDiff.Dual(2,0);
log(a*b + max(a,2))

Dual{Nothing}(2.1972245773362196,0.3333333333333333)

### 2.4.2 Reverse Accumulation

In [5]:
import Zygote: gradient
f(a, b) = log(a*b + max(a,2));
gradient(f, 3.0, 2.0)

(0.3333333333333333, 0.3333333333333333)